# 포트폴리오 최적화: 세 가지 방법론 비교

이 노트북은 MPT, Risk Parity, Kelly Criterion 세 가지 포트폴리오 최적화 방법을 비교합니다.

## 1. 포트폴리오 최적화 개요

포트폴리오 최적화는 자산 배분을 결정하는 핵심 과제입니다.

### 비교할 방법론:
1. **Modern Portfolio Theory (MPT)**: Sharpe ratio 최대화
2. **Risk Parity**: 각 자산의 위험 기여도 균등
3. **Kelly Criterion**: 기대 수익에 따른 동적 배분

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Import from crypto-lab
from src.risk.portfolio_optimization import (
    calculate_risk_parity_weights,
    optimize_kelly_criterion,
    optimize_portfolio_mpt,
)

# Set up visualization
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

print("✓ 환경 설정 완료")

## 2. 데이터 준비

포트폴리오에 포함할 자산의 수익률 데이터를 준비합니다.

In [ ]:
# Example: Create sample returns for demonstration
# In practice, you would load real market data

# Assets: BTC, ETH, SOL, ADA
assets = ["BTC", "ETH", "SOL", "ADA"]
n_assets = len(assets)
n_periods = 252  # 1 year of daily data

# Generate correlated returns
np.random.seed(42)
returns_data = pd.DataFrame(
    np.random.multivariate_normal(
        [0.0005, 0.0004, 0.0003, 0.0002],
        [
            [0.0004, 0.0002, 0.0001, 0.0001],
            [0.0002, 0.0003, 0.0001, 0.00005],
            [0.0001, 0.0001, 0.0005, 0.00008],
            [0.0001, 0.00005, 0.00008, 0.0002],
        ],
        n_periods,
    ),
    columns=assets,
)

# Calculate statistics
cov_matrix = returns_data.cov()
corr_matrix = returns_data.corr()
mean_returns = returns_data.mean()
std_returns = returns_data.std()

print(f"포트폴리오 자산: {assets}")
print("\n기대 연간 수익률:")
print((mean_returns * 252).to_string())
print("\n연간 변동성:")
print((std_returns * np.sqrt(252)).to_string())

## 3. Modern Portfolio Theory (MPT)

Sharpe ratio를 최대화하는 최적 포트폴리오를 찾습니다.

In [ ]:
# Optimize for maximum Sharpe ratio
weights_mpt = optimize_portfolio_mpt(
    mean_returns=mean_returns, cov_matrix=cov_matrix, risk_free_rate=0.03
)

# Calculate portfolio metrics
portfolio_return_mpt = np.sum(mean_returns * weights_mpt) * 252
portfolio_vol_mpt = np.sqrt(np.dot(weights_mpt, np.dot(cov_matrix, weights_mpt))) * np.sqrt(252)
sharpe_ratio_mpt = (portfolio_return_mpt - 0.03) / portfolio_vol_mpt

print("Modern Portfolio Theory (MPT)")
print("=" * 50)
for asset, weight in zip(assets, weights_mpt, strict=False):
    print(f"{asset:6s}: {weight:7.2%}")
print(f"\n포트폴리오 수익률: {portfolio_return_mpt:.2%}")
print(f"포트폴리오 변동성: {portfolio_vol_mpt:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio_mpt:.2f}")

## 4. Risk Parity

각 자산의 위험 기여도를 균등하게 하는 방식입니다.

In [ ]:
# Calculate Risk Parity weights
weights_rp = calculate_risk_parity_weights(cov_matrix)

# Calculate portfolio metrics
portfolio_return_rp = np.sum(mean_returns * weights_rp) * 252
portfolio_vol_rp = np.sqrt(np.dot(weights_rp, np.dot(cov_matrix, weights_rp))) * np.sqrt(252)
sharpe_ratio_rp = (portfolio_return_rp - 0.03) / portfolio_vol_rp

print("Risk Parity")
print("=" * 50)
for asset, weight in zip(assets, weights_rp, strict=False):
    print(f"{asset:6s}: {weight:7.2%}")
print(f"\n포트폴리오 수익률: {portfolio_return_rp:.2%}")
print(f"포트폴리오 변동성: {portfolio_vol_rp:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio_rp:.2f}")

## 5. Kelly Criterion

장기 성장을 최대화하는 동적 배분 방식입니다.

In [ ]:
# Calculate Kelly weights
# Kelly Criterion: f* = (p*b - q) / b
# Where p = win probability, q = loss probability, b = odds

weights_kelly = optimize_kelly_criterion(mean_returns=mean_returns, cov_matrix=cov_matrix)

# Calculate portfolio metrics
portfolio_return_kelly = np.sum(mean_returns * weights_kelly) * 252
portfolio_vol_kelly = np.sqrt(np.dot(weights_kelly, np.dot(cov_matrix, weights_kelly))) * np.sqrt(
    252
)
sharpe_ratio_kelly = (portfolio_return_kelly - 0.03) / portfolio_vol_kelly

print("Kelly Criterion")
print("=" * 50)
for asset, weight in zip(assets, weights_kelly, strict=False):
    print(f"{asset:6s}: {weight:7.2%}")
print(f"\n포트폴리오 수익률: {portfolio_return_kelly:.2%}")
print(f"포트폴리오 변동성: {portfolio_vol_kelly:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio_kelly:.2f}")

## 6. 세 가지 방법론 비교

In [ ]:
# Create comparison table
comparison_data = {
    "방법론": ["MPT", "Risk Parity", "Kelly Criterion"],
    "수익률": [
        f"{portfolio_return_mpt:.2%}",
        f"{portfolio_return_rp:.2%}",
        f"{portfolio_return_kelly:.2%}",
    ],
    "변동성": [f"{portfolio_vol_mpt:.2%}", f"{portfolio_vol_rp:.2%}", f"{portfolio_vol_kelly:.2%}"],
    "Sharpe Ratio": [
        f"{sharpe_ratio_mpt:.2f}",
        f"{sharpe_ratio_rp:.2f}",
        f"{sharpe_ratio_kelly:.2f}",
    ],
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Weights comparison
weights_df = pd.DataFrame(
    {"MPT": weights_mpt, "Risk Parity": weights_rp, "Kelly": weights_kelly}, index=assets
)

weights_df.plot(kind="bar", ax=axes[0])
axes[0].set_title("포트폴리오 구성 비교")
axes[0].set_ylabel("가중치")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Efficient Frontier
portfolios = np.random.multivariate_normal(mean_returns, cov_matrix, 1000)
returns_plot = []
vols_plot = []
for _ in range(1000):
    w = np.random.dirichlet(np.ones(n_assets))
    returns_plot.append(np.sum(mean_returns * w) * 252)
    vols_plot.append(np.sqrt(np.dot(w, np.dot(cov_matrix, w))) * np.sqrt(252))

axes[1].scatter(vols_plot, returns_plot, alpha=0.3, s=10)
axes[1].scatter(
    portfolio_vol_mpt, portfolio_return_mpt, marker="*", s=500, label="MPT", color="red"
)
axes[1].scatter(
    portfolio_vol_rp, portfolio_return_rp, marker="s", s=200, label="Risk Parity", color="green"
)
axes[1].scatter(
    portfolio_vol_kelly, portfolio_return_kelly, marker="^", s=200, label="Kelly", color="blue"
)
axes[1].set_xlabel("변동성")
axes[1].set_ylabel("기대 수익률")
axes[1].set_title("효율적 프론티어")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 비교 분석 완료")

## 7. 거래 비용 고려

In [ ]:
# Consider transaction costs
transaction_cost_rate = 0.0005  # 0.05%

# Rebalancing frequency: quarterly
rebalancing_per_year = 4
annual_transaction_cost_mpt = weights_mpt.sum() / 2 * transaction_cost_rate * rebalancing_per_year
annual_transaction_cost_rp = weights_rp.sum() / 2 * transaction_cost_rate * rebalancing_per_year
annual_transaction_cost_kelly = (
    weights_kelly.sum() / 2 * transaction_cost_rate * rebalancing_per_year
)

# Net returns after costs
net_return_mpt = portfolio_return_mpt - annual_transaction_cost_mpt
net_return_rp = portfolio_return_rp - annual_transaction_cost_rp
net_return_kelly = portfolio_return_kelly - annual_transaction_cost_kelly

print("거래 비용 고려 후 수익률")
print("=" * 50)
print(f"MPT: {portfolio_return_mpt:.2%} - {annual_transaction_cost_mpt:.2%} = {net_return_mpt:.2%}")
print(
    f"Risk Parity: {portfolio_return_rp:.2%} - {annual_transaction_cost_rp:.2%} = {net_return_rp:.2%}"
)
print(
    f"Kelly: {portfolio_return_kelly:.2%} - {annual_transaction_cost_kelly:.2%} = {net_return_kelly:.2%}"
)

## 8. 결론

### 각 방법론의 특징:

**MPT (Modern Portfolio Theory)**
- 장점: Sharpe ratio 최적화, 수학적으로 엄밀함
- 단점: 과거 데이터에 의존, 극단적인 가중치 가능

**Risk Parity**
- 장점: 균형잡힌 위험 배분, 구현이 간단
- 단점: 수익률 목표가 없음, 상관관계 변화에 취약

**Kelly Criterion**
- 장점: 장기 성장 최대화, 동적 적응
- 단점: 변동성이 높을 수 있음, 추정치에 민감

### 실무 권장사항:

1. **포트폴리오 규모**: 작을수록 Kelly, 클수록 MPT
2. **리스크 선호도**: 보수적이면 Risk Parity, 공격적이면 Kelly
3. **운영 환경**: 복잡할수록 MPT, 간단할수록 Risk Parity
4. **하이브리드 접근**: 여러 방법을 결합하여 사용